# Metabolome and Lipidome to predict GDM
## Data Exploration

In [ ]:
import pandas as pd
import numpy as np

from sklearn.linear_model import ElasticNet

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
ls

## 1. Lipidomics

In [ ]:
xls = pd.ExcelFile('gdm_multiomics_original_datasets/Hurtado_mx_898628_human_plasma_lipidomics_11-2025_submit.xlsx')
print(xls.sheet_names)


In [ ]:
lip_df = pd.read_excel('gdm_multiomics_original_datasets/Hurtado_mx_898628_human_plasma_lipidomics_11-2025_submit.xlsx',sheet_name='Data')

In [ ]:
lip_df.head(10)

In [ ]:
## split the dataframe into (1)annotation, (2)samples
# annot_df = lip_df.loc[:,:'Unnamed: 6']
annot_df = lip_df[['Unnamed: 2']]
sample_df = lip_df.loc[:,'MHCMFM1-41':]

## concat annot_df and sample_df
annot_sample_df = pd.concat([annot_df,sample_df],axis=1)


In [ ]:
##select relevant rows & divide them
dx_df = pd.DataFrame(annot_sample_df.loc[3,:]).T
sample_df=annot_sample_df.loc[8:,:]

##concat dianosis + LC-MS samples
dx_sample_df = pd.concat([dx_df,sample_df],axis=0)

## reindexing after the concating
dx_sample_df = dx_sample_df.reset_index().drop(columns='index')

In [ ]:
dx_sample_df

Summary of lipidomics data:
1. 1st column: annotation of the samples
2. 1st row: GDM diagnosis
3. NAs in the 1st column: (1)analyze it only with annotated lipidomics first, (2) then include unannotated ones
4. NAs in the 1st row: Patients without GDM dianosis results - analyze it only with the results

In [ ]:
dx_sample_df.to_csv('gdm_multiomics_cleaned_datasets/gdm_lipidomics_cleaned_df.csv')

## 2. Metabolomics-(1)